# Part I

In [9]:
!pip install transformers[torch] datasets evaluate seqeval

## Load data

In [2]:
from datasets import load_dataset

In [27]:
wnut = load_dataset("wnut_17")

In [7]:
# wnut["train"][0]

## Preprocess

In [28]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

/tmp/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [29]:
example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens[:10]

['[CLS]', '@', 'paul', '##walk', 'it', "'", 's', 'the', 'view', 'from']

In [30]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [31]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

In [32]:
tokenized_wnut["train"][0]

{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'input_ids': [101,
  1030,
  2703,
  17122,
  2009,
  1005,
  1055,
  1996,
  3193,
  2013,
  2073,
  1045,
  1005,
  1049,
  2542,
  2005,
  2048,
  3134,
  1012,
  3400,
  2110,
  2311,
  1027,
  9686,
  2497,
  1012,
  3492,
  2919,
  4040,
  2182,
  2197,
  3944,
  1012,
  102],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [-100,
  0,
  -100,
  -100,
  0,
  0,
  -100,
  0,
  0,
  0,
  0,
  0,
  0,
  

In [33]:
from transformers import DataCollatorForTokenClassification

# data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

## Evaluate

In [34]:
import evaluate

seqeval = evaluate.load("seqeval")

In [35]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

In [36]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [38]:
example['ner_tags']

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 7,
 8,
 8,
 0,
 7,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [24]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
}
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

## Train

In [6]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

# model = AutoModelForTokenClassification.from_pretrained(
#     "distilbert-base-uncased", num_labels=13, id2label=id2label, label2id=label2id
# )

In [70]:
example = tokenized_wnut["train"][0]
# x = torch.Tensor(example["input_ids"]).unsqueeze(0).type(torch.int).to(device)
# x.shape
# pred = model(x)
pred.logits.shape

torch.Size([1, 34, 13])

In [7]:
import torch

device = torch.device('cuda')
# model = model.to(device)

In [8]:
training_args = TrainingArguments(
    output_dir="my_awesome_wnut_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_wnut["train"],
    eval_dataset=tokenized_wnut["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

NameError: name 'model' is not defined

## Inference

In [43]:
text = "The Golden State Warriors are an American professional basketball team based in San Francisco."

In [40]:
from transformers import pipeline

classifier = pipeline("ner", model="/content/my_awesome_wnut_model/checkpoint-426")
results = classifier(text)

OSError: Incorrect path_or_model_id: '/content/my_awesome_wnut_model/checkpoint-426'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

# Part II

In [8]:
bad_words = [
  "вроде",
  "типа",
  "как бы",
  "это самое",
  "короче",
  "на самом деле",
  "собственно",
  "таким образом",
  "как говорится",
  "как его",
  "вообще-то",
  "просто",
  "в общем",
  "типа",
  "в целом",
  "то есть",
  "как бы",
  "блин",
]

In [9]:
raw_dataset = load_dataset("Den4ikAI/russian_dialogues", split='train[0:10000]')

In [10]:
from nltk.tokenize import WordPunctTokenizer

wptokenizer = WordPunctTokenizer()

raw_dataset = [wptokenizer.tokenize(line.lower()) for line in raw_dataset["question"] + raw_dataset["answer"] if line is not None]

In [11]:
import pandas as pd

ner_dataset = []
for sentence in raw_dataset:
    if sentence is None:
        continue
    ner_tags = []
    for word in sentence:
        ner_tag = int(word in bad_words)
        ner_tags.append(ner_tag)
    ner_dataset.append([sentence, ner_tags])

ner_dataset = pd.DataFrame(ner_dataset).sample(frac=1, random_state=42).reset_index()
ner_dataset.columns = ["id", "tokens", "ner_tags"]

In [12]:
from datasets import Dataset

bad_dataset = Dataset.from_pandas(ner_dataset)

In [13]:
bad_dataset

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 19996
})

In [14]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_bad_dataset = bad_dataset.map(tokenize_and_align_labels, batched=True)


/tmp/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Map:   0%|          | 0/19996 [00:00<?, ? examples/s]

In [15]:
tokenized_bad_dataset[0]

{'id': 18883,
 'tokens': ['тамне', 'знаю'],
 'ner_tags': [0, 0],
 'input_ids': [2, 5219, 987, 30310, 3],
 'token_type_ids': [0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1],
 'labels': [-100, 0, -100, 0, -100]}

In [16]:
model = AutoModelForTokenClassification.from_pretrained(
    "cointegrated/rubert-tiny2", num_labels=2, id2label={0:'0', 1:'1'}, label2id={'0':0, '1':1}
)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
model = model.to('cuda')

In [19]:
import os

os.environ['WANDB_DISABLED'] = 'true'

In [ ]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [39]:
#TODO training

training_args = TrainingArguments(
    output_dir="my_awesome_bad_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    # push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_bad_dataset,
    eval_dataset=tokenized_bad_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Epoch,Training Loss,Validation Loss
1,0.000100,0.000063
2,0.000100,0.000049


TrainOutput(global_step=2500, training_loss=0.00016218241602182387, metrics={'train_runtime': 50.1736, 'train_samples_per_second': 797.072, 'train_steps_per_second': 49.827, 'total_flos': 20733231750960.0, 'train_loss': 0.00016218241602182387, 'epoch': 2.0})

In [ ]:
text = 'то, как мы его воспринимаем, нравится он нам или нет, кажется он из клявым или набород бархатным глубоким и красивым. Это наверное то, что больше всего влияет на нашего сприятия голоса того или иного человека. Не женит, и именно поэтому сегодняшним выпуске, мы вместе с вами постараемся ответить на вопросы, можно ли изменить тот тембр голоса, который данном, при рождения и если можно, то как его лучшеть, как сделать все более красивым и приятным. Скубничкой. Начнем с ответы на первый вопрос, можем ли мы изменить тот тембр голоса, который данном при рождения. Для этого нам естественно необходимо разобраться в процессе звуку излечения, то есть как вообще формируется на штембр голос. На картинке вы видите, важнейшую часть нашего голосового аппарата, который называется голосовые складки. Именно благодаря колебанию этих складок и рождается звуковая волна, которая.
'

In [41]:
model_classifier = pipeline('ner', model='../checkpoint-2500/')

/tmp/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


OSError: checkpoint-2500 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`